In [ ]:
import pandas as pd
import numpy as np

import random

pd.set_option('display.max_rows', 50)


from functions import get_tickers_stocks, get_close_prices, double_listed_stocks, sharpe_ratio_calculation, generate_rand_portfolios, select_top_five

In [ ]:
#GET THE STOCKS

us_exchanges = ['NMS', 'NYQ', 'NGM']
eu_exchanges = ['PAR', 'FRA', 'LSE', 'AMS']
asia_exchanges = ['SHH', 'JPX', 'HKG']

selected_exchanges = us_exchanges + eu_exchanges + asia_exchanges

full_selected_stocks = {}
df_all_stocks = pd.DataFrame()
for exchange in selected_exchanges:
    print(f'Extracting from {exchange}')
    exchanges = [exchange]
    selected_stocks_dict, ticker_list = get_tickers_stocks(50000, exchanges, 50)

    full_selected_stocks.update(selected_stocks_dict)

    if len(ticker_list) > 0: 
        print('YES')
        df = get_close_prices(ticker_list, period = 2, start = '2022-01-01')
        df_all_stocks = pd.concat([df_all_stocks, df], axis=1)

doubly_listed_tickers = double_listed_stocks(full_selected_stocks)

for ticker_to_drop in doubly_listed_tickers:
    try:
        df_all_stocks = df_all_stocks.drop(columns=[ticker_to_drop])
    except:
        pass


df_all_stocks = df_all_stocks.ffill() #ffill again after concatenating the tickers

In [ ]:
#GETTING CRYPTOS FROM COINBASE 50 INDEX
#https://www.marketvector.com/factsheets/download/COIN50.d.pdf

coinbase_50_cryptos = ['BTC', 'ETH', 'XRP', 'SOL', 'DOGE', 'ADA', 'LINK', 'XLM', 'AVAX', 'SHIB', 'DOT', 'LTC', 'BCH', 
                       'UNI', 'NEAR', 'PEPE', 'APT', 'ICP', 'ETC', 'AAVE', 'RNDR', 'ATOM', 'MATIC', 'ALGO', 'EOS', 'MKR', 
                       'ASI', 'QNT', 'BONK', 'STX', 'INJ', 'GRT', 'LDO', 'XTZ', 'CRV', 'SAND', 'ZEC', 'HNT', 'JASMY', 'MANA', 
                       'AXS', 'WIF', 'CHZ', 'COMP', 'APE', 'AERO', '1INCH', 'SNX', 'ROSE', 'LPT']

crypto_tickers_fixed = [tick + "-USD" for tick in coinbase_50_cryptos]

cryptos_df = get_close_prices(crypto_tickers_fixed, period = 2, start = '2022-01-01')

print('Number of NA values in cryptos_df is ', df.isna().any().sum())

#cryptos_df

In [ ]:
#Random Portfolios generation

tickers = list(df_all_stocks.columns)

random.seed(42)
random_portfolios = generate_rand_portfolios(n_reps=1000, n_stocks=15, tickers=tickers)

In [ ]:
#Select top five sharpe ratio portfolios from a portfolio

sharpe_ratio = sharpe_ratio_calculation(df_all_stocks, rf_rate_annual = 0.02)

top_five_dict = select_top_five(random_portfolios, metric=sharpe_ratio)

In [ ]:
# df_pct_change = df_all_stocks.pct_change().dropna()
# cov_matrix = df_pct_change.cov().values

In [ ]:
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns


mu = expected_returns.mean_historical_return(df_all_stocks)  # Expected returns
S = risk_models.sample_cov(df_all_stocks)  # Covariance matrix
#S = risk_models.CovarianceShrinkage(df_all_stocks).ledoit_wolf()

ef = EfficientFrontier(mu, S, weight_bounds=(0,1))


#ADD CONSTRAINTS
ef.add_constraint(lambda w: w[ef.tickers.index('AAPL')] >= 0.10)
ef.add_constraint(lambda w: w[ef.tickers.index('GOOG')] >= 0.05)


weights = ef.min_volatility()


# print("Optimal portfolio weights (min variance):")
# for stock, weight in weights.items():
#     if weight != 0:
#         print(f"{stock}: {weight:.4f}")

In [ ]:
# import cvxpy as cp

# ef = EfficientFrontier(mu, S, solver=cp.GUROBI)
# booleans = cp.Variable(len(ef.tickers), boolean=True)
# ef.add_constraint(lambda x: x <= booleans)
# ef.add_constraint(lambda x: cp.sum(booleans) <= 10)
# ef.min_volatility()

In [ ]:
from scipy.optimize import minimize

returns = df_all_stocks.pct_change().dropna()
cov_matrix = returns.cov()




expected_returns = returns.mean()

# Define the optimization function (objective: minimize portfolio variance)
def objective(weights):
    portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))
    return portfolio_variance

# Constraints: 
# - The portfolio should contain exactly 15 stocks (5 preselected + 10 chosen)
# - The weights should sum to 1 (fully invested portfolio)
def constraints(weights):
    return np.sum(weights) - 1  # sum of weights must be 1

# Bounds for each stock's weight (between 0 and 1)
bounds = [(0, 1) for _ in range(len(returns))]

# Initial guess: Equal weight for each stock
initial_guess = np.ones(len(returns)) / len(returns)

# Define the optimization problem
result = minimize(
    objective, initial_guess, method='SLSQP', bounds=bounds, constraints={'type': 'eq', 'fun': constraints}
)

# Check if the optimization was successful
if result.success:
    optimized_weights = result.x
    selected_stocks = investment_universe
    stock_weights = dict(zip(selected_stocks, optimized_weights))

    # Get the 5 preselected stocks' weights
    preselected_weights = {stock: stock_weights[stock] for stock in preselected_stocks}
    print("Preselected Stock Weights:")
    print(preselected_weights)
    
    # Get the 10 new stocks' weights (excluding preselected)
    remaining_stocks = [stock for stock in selected_stocks if stock not in preselected_stocks]
    remaining_weights = {stock: stock_weights[stock] for stock in remaining_stocks}
    print("\nRemaining Stock Weights:")
    print(remaining_weights)
else:
    print("Optimization failed:", result.message)